In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from hails.hails import HAILS_Univ
from hails.seq_layers import DLinear

In [3]:
NUM_NODES = 1_000
BATCH = 32
TIME = 100
PRED_LEN = 30

In [4]:
input_time_series = torch.randn(BATCH, TIME, NUM_NODES)
forecasting_model = DLinear

In [5]:
hails_model = HAILS_Univ(NUM_NODES, TIME, PRED_LEN, forecasting_model)

In [6]:
y_base = hails_model._forward_base(input_time_series)
y_base_mu, y_base_logsigma = y_base

In [7]:
print(y_base_mu.shape, y_base_logsigma.shape)

torch.Size([32, 30, 1000]) torch.Size([32, 30, 1000])


In [8]:
y_mu, y_logstd = hails_model.forward(input_time_series)

In [9]:
print(y_mu.shape, y_logstd.shape)

torch.Size([32, 30, 1000]) torch.Size([32, 30, 1000])


In [10]:
# Identity matric od size NUM_NODES
y_variance = torch.exp(y_logstd * 2)
hmatrix = torch.eye(NUM_NODES)
y_mu_1, y_variance_1 = hails_model._get_agg_params(y_mu, y_variance, hmatrix)
# The result should be the same as y_mu and y_logstd
print(torch.allclose(y_mu, y_mu_1))
print(torch.allclose(y_variance, y_variance_1))

True
True


In [11]:
print(hails_model._dch_normal_loss(y_mu, y_variance, hmatrix).mean())

tensor(0., grad_fn=<MeanBackward0>)


In [12]:
print(hails_model._dch_poisson_loss(y_mu.exp(), hmatrix).mean())

tensor(0., grad_fn=<MeanBackward0>)


In [14]:
dist_mask = (torch.rand(NUM_NODES) > 0.5).float()
loss = hails_model.get_jsd_loss(y_mu, y_logstd, hmatrix, dist_mask)

In [15]:
print(loss.mean())

tensor(0., grad_fn=<MeanBackward0>)
